In [1]:
import numpy as np
import random
import math

## Аналитический метод

In [2]:
def solve_analytical(game_params):
    r = game_params['r'] / math.sqrt(2)
    return game_params['n'] / 2 * (1 - math.sqrt(1 - (r / game_params['R'])**2))

## Равномерная случайная точка на сфере

In [3]:
def rand_sphere_point(R = 1):
    y = random.uniform(-1, 1)
    alpha = random.uniform(0, 2 * math.pi)
    
    r = math.sqrt(1 - y**2) 
    
    x = r * math.cos(alpha)
    z = r * math.sin(alpha)
    
    return (R * x, R * y, R * z)

## Визуальная проверка равномерности распределения

In [4]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

data = np.array([rand_sphere_point() for i in range(5000)])

trace1 = go.Scatter3d(
    x=data[:,0],
    y=data[:,1],
    z=data[:,2],
    mode='markers',
    marker=dict(
        size=5,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=1
        ),
        opacity=0.8
    )
)
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=[trace1], layout=layout)
iplot(fig, filename='simple-3d-scatter')

## Численное решение

In [5]:
TESTS_COUNT = 10000

def distane(lhs, rhs):
    if len(rhs) != len(rhs) or len(lhs) < 1:
        raise Exception('Cant aclculate distance')
        
    summ = 0
    
    return math.sqrt(sum((np.array(lhs) - np.array(rhs))**2))

def solve_iterative(game_params):
    K = 0
    
    R = game_params['R']
    r = game_params['r']
    n = game_params['n']
    
    for i in range(TESTS_COUNT):
        finder_points = [rand_sphere_point(R) for i in range(n)]
        hider_point = rand_sphere_point(R)
        
        for p in finder_points:
            if (distane(p, hider_point)) <= r:
                K += 1
                break
    
    return K / TESTS_COUNT

## Конкретные параметры

In [6]:
game_params = {
    'R' : 1,
    'r' : math.sqrt(2),
    'n' : 1
}
print(solve_iterative(game_params))
print(solve_analytical(game_params))

0.5092
0.5
